# Другой подход к классификации

In [206]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("/content/drive/MyDrive/intern_task_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/intern_task_test.csv")

In [207]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import accuracy_score


def NDCG_atK_score(test_Data, pred_Data, k, logits=False):
  query_ids = test_Data['query_id'].unique()
  test_Data_ranks = np.asarray([[test_Data[test_Data['query_id'] == id]['rank'].iloc[i] for i in range(min(len(test_Data[test_Data['query_id'] == id]), k))] + [0 for _ in range(max(0, k - len(test_Data[test_Data['query_id'] == id])))] for id in query_ids])
  pred_Data_ranks = np.asarray([[pred_Data[pred_Data['query_id'] == id]['rank'].iloc[i] for i in range(min(len(pred_Data[pred_Data['query_id'] == id]), k))] + [0 for _ in range(max(0, k - len(pred_Data[pred_Data['query_id'] == id])))] for id in query_ids])
  if logits:
    print(test_Data_ranks)
    print(pred_Data_ranks)
  return ndcg_score(test_Data_ranks, pred_Data_ranks)

В этом подходе попробуем использовать факт упорядоченности классов ранга. Разделим на хорошие и плохие. Пусть, если объект имеет ранг 3 или 4, то соответсвующая метка 'good' будет иметь значение 1. Иначе - 0.

Таким образом, мы сможем одним классификатором отделить хорошие объекты от плохих. Этот подход хорош тем, что во-первых, классификатору будет легче определить хороший класс или плохой, чем сравнивать очень близкие по классу объекты, во-вторых, учитывается упорядоченность.

Кроме того, факт того, что объект имеет метку 1 в 'good' и находится далеко от разделяющей гиперплоскости, поможет отделить 4 от 3. Аналогично при метке 0.

Добавим в датафрейм столбец 'good':

In [208]:
df_train['good'] = (df_train['rank'] > 2).astype(int)

Создадим класс для реализации идеи.

Таким образом в данном классе классификатор учится отделять хорошие объекты от плохих. А затем преобразует полученные значения в метку 'rank'.

Если вероятность объекта принадлежать к хорошему классу ближе к 1, чем к 0.5, то объекту присваивается метка 4. Иначе - 3.

Аналогично, для плохих меток.

Если же классификатор не может уверенно определить объект "хороший" или "плохой", то данному объекту присвоится метка 2.

In [209]:
from sklearn.linear_model import LogisticRegression


class Classifier():
  def __init__(self):
    self.estimator_good = LogisticRegression(max_iter=1500, class_weight='balanced')
    self.EPS = 0.1

  def fit(self, X, Y):
    self.estimator_good.fit(X, Y['good'])

  def predict(self, X):
    self.y_pred_good = self.estimator_good.predict_proba(X)
    return self.transform()


  def transform(self):
    y_pred_good = self.y_pred_good
    output = []
    for i in range(len(y_pred_good)):
      if abs(y_pred_good[i][1] - y_pred_good[i][0]) < self.EPS:
        output.append(2)
      elif y_pred_good[i][1] > 0.5:
        if abs(1 - y_pred_good[i][1]) < abs(0.5 - y_pred_good[i][1]):
          output.append(4)
        else:
          output.append(3)
      else:
        if abs(1 - y_pred_good[i][0]) < abs(0.5 - y_pred_good[i][0]):
          output.append(0)
        else:
          output.append(1)
    return np.array(output)


In [210]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

to_drop = ['feature_14', 'feature_17', 'feature_19', 'feature_20',
       'feature_29', 'feature_34', 'feature_35', 'feature_44',
       'feature_74', 'feature_79', 'feature_84', 'feature_89',
       'feature_94', 'feature_114', 'feature_119', 'feature_120',
       'feature_121', 'feature_122', 'feature_123', 'feature_124',
       'feature_142', 'feature_143']


cat_features = ['feature_95', 'feature_96', 'feature_97', 'feature_98', 'feature_99']

df_train = pd.get_dummies(df_train, columns=cat_features)
df_test = pd.get_dummies(df_test, columns=cat_features)

X_train = df_train.drop(columns=to_drop)
X_train = X_train.drop(columns=['rank', 'query_id', 'good'])
X_test = df_test.drop(columns=to_drop)
X_test = X_test.drop(columns=['rank', 'query_id'])
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [211]:
est = Classifier()
est.fit(X_train, df_train)

In [212]:
y = est.predict(X_test)

In [213]:
df_tmp = df_test.copy()
df_tmp['rank'] = y
print("NDCG@5 score : ", NDCG_atK_score(df_test, df_tmp, k=5))
print("NDCG@10 score : ", NDCG_atK_score(df_test, df_tmp, k=10))

NDCG@5 score :  0.914988473160503
NDCG@10 score :  0.8892809409107936


In [214]:
NDCG_atK_score(df_test, df_tmp, k=10, logits=True)

[[1 1 1 ... 1 1 1]
 [4 3 2 ... 2 2 2]
 [4 3 2 ... 1 1 1]
 ...
 [4 4 4 ... 4 4 4]
 [3 3 3 ... 2 2 2]
 [3 2 2 ... 1 1 1]]
[[0 1 0 ... 0 1 1]
 [3 1 2 ... 3 1 1]
 [3 3 1 ... 1 0 1]
 ...
 [3 3 3 ... 3 3 1]
 [4 3 3 ... 4 4 3]
 [2 1 1 ... 3 2 3]]


0.8892809409107936